In [1]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from typing import Iterable, List
from model import Transformer
from data import fr_to_en
import utils
import torch.nn as nn
import pandas as pd
import json
import torch



### Vocab 만들기

In [2]:
# 훈련 데이터 불러오기
# Fr -> En 번역을 위한 데이터셋(Multi-30k) 활용
fr_train = utils.open_text_set("data/training/train.fr")
en_train = utils.open_text_set("data/training/train.en")

# Vocab 만들기 / 관련 함수는 utils.py 참조
try : 
  vocab_transform, token_transform = utils.make_vocab(fr_train, en_train)
except :  
  # 오류 발생 시 spacy 설치 필요

  # spacy tokenizer 다운로드(en,fr)
  import spacy.cli
  spacy.cli.download("en_core_web_sm")
  spacy.cli.download("fr_core_news_sm")
  vocab_transform, token_transform = utils.make_vocab(fr_train, en_train)

# param
SRC_LANGUAGE = "fr"
TGT_LANGUAGE = "en"

### 학습한 모델 불러오기

In [3]:
with open('config/transformer.json', 'r') as file:
    param = json.load(file)
    print('Model_Parameters')
    print('-'*50)
    print(param)  

# multi-30k 데이터를 20번 epoch한 모델
model = Transformer(**param)

# model 불러오기
model.load_state_dict(torch.load('model/model.pth'))

# 모델 평가모드로 변경
model.eval()


device = model.device

print('-'*50)
print(f'현재 devicde 설정값은 : "{model.device}" 입니다. 변경을 희망하실 경우 config/transformer.json을 수정해주세요.')
print('-'*50)

# loss_fn
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=1)


Model_Parameters
--------------------------------------------------
{'src_vocab_size': 11509, 'trg_vocab_size': 10837, 'src_pad_idx': 1, 'trg_pad_idx': 1, 'embed_size': 512, 'num_layers': 3, 'forward_expansion': 2, 'heads': 8, 'dropout': 0.1, 'device': 'cpu', 'max_length': 140}
--------------------------------------------------
현재 devicde 설정값은 : "cpu" 입니다. 변경을 희망하실 경우 config/transformer.json을 수정해주세요.
--------------------------------------------------


### 모델 성능 테스트

* validation은 문제와 정답이 모두 주어진다면 test는 문제만 주는 상황임.

* test 함수를 통해 Transformer의 실제 문제 예측 과정을 이해할 수 있음.

* Transformer는 문제와 정답이 있다면 답을 구하는 과정을 병렬적으로 수행할 수 있음.

* 하지만 테스트에서는 정답이 주어지지 않으므로 한 번의 하나의 토큰을 생산함.

* < bos > token을 시작으로 다음 토큰을 예상하며 < eos > 토큰이 생성될때까지 반복적으로 예측을 수행하게 되는 알고리즘이 필요함.

* 아래의 test 함수를 다뤄보면서 Transformer의 데이터 처리 과정을 이해할 수 있음.


In [5]:
# token을 단어로 바꾸기 위한 dict 생성, vocab의 key와 value 위치 변경
# 아래 helper 함수에서 활용됨.
decoder_en = {v:k for k,v in vocab_transform['en'].get_stoi().items()}
decoder_fr = {v:k for k,v in vocab_transform['fr'].get_stoi().items()}


def helper_what_sen(src,trg,logits,i,c=100,sen_num=0) : 
    '''
    문장이 제대로 학습되고 있는지를 확인하는 함수

    src = encoding 된 source_sentence 
    trg = encoding 된 target_sentence
    logits = 모델 예측값
    i = 현재 batch 순서
    c = 결과를 보여주는 단계, ex) c = 100이면 100,200,300... 번째 batch에서 결과를 보여줌
    sen_num = batch 내 문장 중 몇 번째 문장을 추적할 것인지 설정
    '''
    if i % c == 0 and i != 0 :
        src_sen = ' '.join([decoder_fr[i] for i in src.tolist()[sen_num] if decoder_fr[i][0] != '<' ])
        trg_sen = ' '.join([decoder_en[i] for i in trg.tolist()[sen_num] if decoder_en[i][0] != '<' ])
        prediction = logits.max(dim=-1, keepdim=False)[1][sen_num]
        prd_sen = ' '.join([decoder_en[i] for i in prediction.tolist() if decoder_en[i] != '<' ])
        '''
        /*/* 모델의 예측 문장(prd_sen)을 구하는 방법 /*/* 

        n = batch size, trg_token_len = batch 내 문장의 최대 토큰 개수

        모델 output(=logits)은 (n, trg_token_len, trg_vocab_len)의 3차원 텐서임.

        1. 해당 텐서를 trg_vocab_len 차원의 기준으로 max를 하면 (n,trg_token_len)을 반환
        2. tensor.max()의 수행 결과는 [최댓값,idx]를 반환함.
        3. [1]을 넣어 idx를 선택, 그 결과는 (n, trg_token_len) 차원의 idx 반환
        4. 원하는 문장 순서(sen_num)을 선택한 뒤 정수를 다시 단어로 decoding 수행
        '''

        print('')
        print(f'{i}번째 batch에 있는 {sen_num}번째 문장 예측 결과 확인')
        print('src : ',src_sen)
        print('prd : ',prd_sen)
        print('trg : ',trg_sen)
        print('')

    return None

def tokenizing_src(input_data:str) : 
    # input_data_tokenizing
    token_data = token_transform['fr'](input_data)
    vocab_src = vocab_transform['fr'](token_data)
    tokenized_src = [2] + vocab_src + [3]
    return tokenized_src

def select_random_item() :
    num = torch.randint(1,29000,(1,)).item()

    return fr_train[num], en_train[num]

def test(model) :
    '''
    * validation은 문제와 정답이 모두 주어진다면 test는 문제만 주는 상황임.

    * test 함수를 통해 Transformer의 실제 문제 예측 과정을 이해할 수 있음.

    * Transformer는 문제와 정답이 있다면 답을 구하는 과정을 병렬적으로 수행할 수 있음.

    * 하지만 테스트에서는 정답이 주어지지 않으므로 한 번의 하나의 토큰을 생산함.

    * < bos > token을 시작으로 다음 토큰을 예상하며 < eos > 토큰이 생성될때까지 반복적으로 예측을 수행하게 되는 알고리즘이 필요함.

    * 아래의 test 함수를 다뤄보면서 Transformer의 데이터 처리 과정을 이해할 수 있음.

    '''

    model.eval()

    # 임의의 훈련 데이터 선별
    fr_item, en_item = select_random_item()
    
    print('입력 :', fr_item)

    # Input Data 토크나이징 
    tokenized_input = tokenizing_src(fr_item)
    max_length = int(len(tokenized_input) * 1.2)

    # src Tensor에 Token 저장
    src = torch.LongTensor(tokenized_input).unsqueeze(0).to(device)

    # trg Tensor 생성(1, max_length)
    trg = torch.zeros(1,max_length).type_as(src.data).to(device)

    # src encoding
    enc_src = model.encode(src)

    next_trg = 2 # 문장 시작 <bos> idx

    # 문장 예측 시작
    for i in range(0,max_length) :
        trg[0][i] = next_trg # token 저장

        logits = model.decode(src,trg,enc_src) # output 산출

        prd = logits.squeeze(0).max(dim=-1, keepdim=False)[1] # 예측 단어 중 max 추출
        next_word = prd.data[i] # i 번째 위치한 단어 추출
        next_trg = next_word.item() 
        if next_trg == 3 :
            # <eos> 나오면 종료
            trg[0][i] = next_trg
            break
    
    # <pad> 제거
    if 3 in trg[0] :
        eos_idx = int(torch.where(trg[0] == 3)[0][0])
        trg = trg[0][:eos_idx].unsqueeze(0)
    else :
        pass

    # 번역
    translation = [decoder_en[i] for i in trg.squeeze(0).tolist()]
    print('모델예측 :',' '.join(translation[1:]))
    

    print('정답 :', en_item)
    print('')
    print('주의! 29,000개의 제한된 데이터로 학습을 수행했으므로 완벽한 예측이 불가능함.')



test(model)


입력 : Un adolescent a un anneau argenté saillant de son nez.
모델예측 : A little boy is holding a piece of a large white shirt is holding
정답 : A teenage boy has a silver ring protruding from his nose.

주의! 29,000개의 제한된 데이터로 학습을 수행했으므로 완벽한 예측이 불가능함.


### Validation 테스트

* Transformer는 문제와 정답이 주어지면 병렬 연산이 가능함.

* test 함수와의 차이를 비교할 수 있도록 validation을 포함하였음.


In [7]:
def collate_fn(batch_iter: Iterable):
    """
    Data_loader에서 불러온 데이터를 가공하는 함수
    토크나이징 => encoding => 시작 끝을 의미하는 spectial token(<bos>,<eos>) 추가 순으로 진행
    """
    text_transform = {}
    for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
        text_transform[ln] = utils.sequential_transforms(
            token_transform[ln],  # 토크나이징
            vocab_transform[ln],  # encoding
            utils.tensor_transform, # BOS/EOS를 추가하고 텐서를 생성
        )  
        # sequential_transform, tensor_transform은 utils.py 참고
    
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch_iter:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample))

    # Pad 붙이기
    PAD_IDX = 1
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch.T, tgt_batch.T



def evaluate(model):
    #모델 평가모드 
    model.eval()
    losses = 0
    
    # Load_Dataset
    dataset= fr_to_en(set_type='validation')

    # validation 데이터 불러오기
    batch_size = 128
    val_dataloader = DataLoader(dataset,batch_size,collate_fn=collate_fn)

    for i,(src,tgt) in enumerate(val_dataloader) :
        
        src = src.to(device)
        tgt = tgt.to(device)
        tgt_input = tgt[:,:-1]

        logits = model(src,tgt_input)

        helper_what_sen(src,tgt_input,logits,i,2,0) # 학습상태 확인

        tgt_output = tgt[:,1:]
        print(logits.reshape(-1,logits.shape[-1]).shape)
        print(tgt_output.reshape(-1).shape)
        loss = loss_fn(logits.reshape(-1,logits.shape[-1]),tgt_output.reshape(-1))

        losses += loss.item()

    return losses / len(val_dataloader)



##### Validation으로 테스트하기

### 출력 문장수를 조정하고 싶으면 helper_what_sen의 parameter 수정

val_loss = evaluate(model)

print('*---'*20)
print(f'Val_loss : {val_loss:.3f}')
print('*---'*20)


► Dataset is "validation"
torch.Size([3712, 10837])
torch.Size([3712])
torch.Size([3840, 10837])
torch.Size([3840])

2번째 batch에 있는 0번째 문장 예측 결과 확인
src :  Un chien noir dans l' herbe , tenant un objet en plastique blanc dans sa gueule .
prd :  A black dog is in the grass with a microphone bench guitar in a mouth . <eos> <eos> <eos> . . . . . . . . . . . .
trg :  A black dog standing in some grass holding a white plastic item in its mouth .

torch.Size([3968, 10837])
torch.Size([3968])
torch.Size([3328, 10837])
torch.Size([3328])

4번째 batch에 있는 0번째 문장 예측 결과 확인
src :  Le chien noir saute au-dessus de l' eau vers un frisbee flottant près d' un bateau .
prd :  The black dog and on the beach with the body in . a boat . <eos> . . . . . . . <eos> . . . . . . . . .
trg :  The black dog jumps above the water towards a Frisbee floating near a boat .

torch.Size([4224, 10837])
torch.Size([4224])
torch.Size([3456, 10837])
torch.Size([3456])

6번째 batch에 있는 0번째 문장 예측 결과 확인
src :  Un homme avec un bad